# 🍠 [Day 18](https://adventofcode.com/2019/day/18)

In [1]:
import heapq
import numpy as np
from collections import defaultdict

def parse_maze(inputs):
    maze = [list(line) for line in inputs.splitlines()]
    return np.array(maze)
    
        
def bfs_with_constraints(maze):
    start = np.where(maze == '@')
    num_keys = np.amax(list(map(ord, maze.flatten()))) - ord('a') + 1
    # Save minimum distances from starting point 
    # for every target point x keys combination
    dists = defaultdict(lambda: defaultdict(lambda: float('inf')))
    # Queue
    # cost, num keys left, keys found, current point
    # The heap will prioritize point with low cost 
    # and, at equal cost, highest number of keys
    queue = [(0, num_keys, (), start[0][0], start[1][0])]
    heapq.heapify(queue)
    
    # Iterate
    while len(queue):
        cost, k, keys, x, y = heapq.heappop(queue)
        for (i, j) in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
            # Wall or locked door
            if (maze[i, j] == '#' or 
                (ord('A') <= ord(maze[i, j]) < ord('a') 
                 and maze[i, j].lower() not in keys)):
                continue
            # Pick up a new key
            elif ord(maze[i, j]) >= ord('a') and maze[i, j] not in keys:
                # Update cost for new keys + next state
                new_keys = keys + (maze[i, j],)
                hashkey = tuple(sorted(new_keys))
                if dists[hashkey][(i, j)] > cost + 1:
                    dists[hashkey][(i, j)] = cost + 1
                    # Add to queue if still to be found
                    if k > 1:
                        heapq.heappush(queue, (cost + 1, k - 1, new_keys, i, j))
                    # Otherwise, We found a correct path
                    # And due to the queue ordering (smallest cost first) it is the shortest one
                    else:
                        return cost + 1, new_keys
            # Advance / normal path
            else:
                hashkey = tuple(sorted(keys))
                if dists[hashkey][(i, j)] > cost + 1:
                    dists[hashkey][(i, j)] = cost + 1
                    heapq.heappush(queue, (cost + 1, k, keys, i, j))
    return None

In [2]:
%%time
with open('inputs/day18.txt', 'r') as f:
    inputs = f.read()
    maze = parse_maze(inputs)
    
print("The shortest path takes a total of {0} steps and is "
      "achieved by collecting the keys in order {1}\n".format(
      *bfs_with_constraints(maze)))

The shortest path takes a total of 6162 steps and is achieved by collecting the keys in order ('e', 'f', 'z', 'b', 'i', 'l', 'a', 'u', 'p', 'd', 'c', 'k', 'o', 'r', 's', 'j', 'q', 'm', 't', 'v', 'x', 'h', 'g', 'n', 'y', 'w')

CPU times: user 18.3 s, sys: 168 ms, total: 18.5 s
Wall time: 18.6 s


In [3]:
## The previous implementation does not scale up to part 2 (number of states blows up)
## when considering multiple robots.
## Instead, we consider only the points where keys lie as states
## To help, we precompute disatances from every to key (including the 
## constraints/doors lying on the way)

def precompute_key_to_key(maze):
    """Precompute all distance cost  + constraints required to go from 
    any key to any other"""
    num_keys = np.amax(list(map(ord, maze.flatten()))) - ord('a') + 1
    keys = ['@'] + [chr(ord('a') + i) for i in range(num_keys)]
    keys = ['@'] + sorted(x for x in maze.flatten() if x >= 'a')
    key_to_key = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: float('inf'))))
    
    for k, key in enumerate(keys):
        start = np.where(maze == key)
        dists = np.zeros((maze.shape[0], maze.shape[1])) + float('inf')
        queue = [(0, [], start[0][0], start[1][0])]
        heapq.heapify(queue)

        # Iterate
        while len(queue):
            cost, doors, x, y = heapq.heappop(queue)
            for (i, j) in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
                # Wall or locked door
                if (i < 0 or i >= maze.shape[0] or 
                    j < 0 or j >= maze.shape[1] or 
                    maze[i, j] == '#'):
                    continue
                # Found a key
                elif ord(maze[i, j]) >= ord('a') and maze[i, j] in keys[k + 1:]:
                    if key_to_key[key][maze[i, j]][tuple(sorted(doors))] > cost + 1:
                        key_to_key[key][maze[i, j]][tuple(sorted(doors))] = cost + 1
                        heapq.heappush(queue, (cost + 1, doors, i, j))
                # Advance / normal path
                else:
                    if dists[i, j] > cost + 1:
                        dists[i, j] = cost + 1
                        if ord('A') <= ord(maze[i, j]) < ord('a') and maze[i, j].lower() != key:
                            new_doors = doors + [maze[i, j].lower()]
                            heapq.heappush(queue, (cost + 1, new_doors, i, j))
                        else:
                            heapq.heappush(queue, (cost + 1, doors, i, j))
    # Restrict keys to ones which are reachable
    keys = sorted(list(key_to_key['@'].keys()))
    return keys, {a: {b: dict(key_to_key[a][b]) for b in key_to_key[a]} for a in key_to_key}


def sparse_bfs_with_constraints(maze):
    keys, key_to_key = precompute_key_to_key(maze)
    # Save minimum distances from starting point 
    # for every target point x keys combination
    dists = defaultdict(lambda: float('inf'))
    # Prioritize heap by minimum distance, then number of keys collected
    queue = [(0, len(keys), ('@',))]
    heapq.heapify(queue)
    
    min_dist = float('inf')
    key_order = None
    
    # Iterate
    while len(queue):
        
        cost, k, current_keys = heapq.heappop(queue)
        state = current_keys[-1]
        # Try to reach any uncollected key
        for nxt_state in (set(keys) - set(current_keys)):
            # resolve indexing
            a, b = state, nxt_state
            if state > nxt_state:
                a, b = nxt_state, state
            # if we can pass all doors
            for constraint, steps in key_to_key[a][b].items():
                new_keys = current_keys + (nxt_state,)
                index = tuple(sorted(new_keys))
                if (set(constraint).issubset(current_keys) and 
                    dists[nxt_state, index] > cost + steps):
                    dists[nxt_state, index] = cost + steps
                    if k == 1:
                        if cost + steps < min_dist:
                            min_dist = cost + steps
                            key_order = new_keys
                        continue
                    heapq.heappush(queue, (cost + steps, k - 1, new_keys))
    return min_dist, key_order

In [4]:
%%time
with open('inputs/day18.txt', 'r') as f:
    inputs = f.read()
    maze = parse_maze(inputs)
    
print("The shortest path takes a total of {0} steps and is "
      "achieved by collecting the keys in order {1}\n".format(
      *sparse_bfs_with_constraints(maze)))

The shortest path takes a total of 6162 steps and is achieved by collecting the keys in order ('@', 'e', 'f', 'z', 'i', 'b', 'l', 'a', 'u', 'p', 'o', 'd', 'c', 'k', 'r', 's', 'j', 'q', 'm', 't', 'x', 'v', 'h', 'g', 'n', 'y', 'w')

CPU times: user 3 s, sys: 0 ns, total: 3 s
Wall time: 3 s


In [5]:
def parse_split_maze(inputs):
    maze = parse_maze(inputs)
    start = np.where(maze == '@')
    x, y = start[0][0], start[1][0]
    maze[x-1:x+2, y-1:y+2] = np.array(
        [["@", "#", "@"], ["#", "#", "#"], ["@", "#", "@"]])
    return (maze[:x + 1, :y + 1], maze[x:, :y + 1], 
           maze[:x + 1, y:], maze[x:, y:])


def parallel_sparse_bfs_with_constraints(mazes):
    keys, key_to_key = zip(*[precompute_key_to_key(maze) for maze in mazes])
    all_keys = sorted([x for z in keys for x in z])
    
    # Save minimum distances from starting point 
    # for every target point x keys combination
    dists = defaultdict(lambda: float('inf'))
    # Prioritize heap by minimum distance, then number of keys collected
    queue = [(0, len(all_keys), ('@',), ('@',) * len(mazes))]
    heapq.heapify(queue)
    
    min_dist = float('inf')
    key_order = None
    
    # Iterate
    while len(queue):
        cost, k, current_keys, states = heapq.heappop(queue)
        
        # Try to reach any uncollected key for every root
        for robot in range(len(mazes)):
            state = states[robot]
            for nxt_state in (set(keys[robot]) - set(current_keys)):
                # resolve indexing
                a, b = state, nxt_state
                if state > nxt_state:
                    a, b = nxt_state, state

                # If we can pass the doors
                for constraint, steps in key_to_key[robot][a][b].items():
                    new_keys = current_keys + (nxt_state,)
                    new_states = states[:robot] + (nxt_state,) + states[robot + 1:]
                    index = tuple(sorted(new_keys))
                    if (set(constraint).issubset(current_keys) and 
                        dists[new_states, index] > cost + steps):
                        dists[new_states, index] = cost + steps
                        if k == 1:
                            if cost + steps < min_dist:
                                min_dist = cost + steps
                                key_order = new_keys
                            continue
                        heapq.heappush(queue, (cost + steps, k - 1, new_keys, new_states))
    return min_dist, key_order

In [6]:
%%time
mazes = parse_split_maze(inputs) 

print("The shortest path in the split maze takes a total of {0} steps and is "
      "achieved by collecting the keys in order {1}\n".format(
      *parallel_sparse_bfs_with_constraints(mazes)))

The shortest path in the split maze takes a total of 1556 steps and is achieved by collecting the keys in order ('@', 'e', 'f', 'z', 'i', 'l', 'o', 'd', 'a', 'b', 'u', 'p', 'c', 'k', 'r', 's', 'j', 'q', 'm', 't', 'x', 'v', 'h', 'g', 'n', 'y', 'w')

CPU times: user 10.6 s, sys: 23.6 ms, total: 10.6 s
Wall time: 10.6 s
